In [9]:
import os
import re
import time
import random
from typing import Optional, Set, List, Tuple
from urllib.parse import quote_plus

import pandas as pd
import requests
from bs4 import BeautifulSoup

pd.set_option("display.max_colwidth", 160)
print("Imports ready.")


Imports ready.


In [10]:
# %%
# --- File paths ---
INPUT_PATH   = "csvs/player_info_without_ids.csv"  # your file
OUTPUT_PATH  = "mls_dps_with_tm.csv"
ENCODING     = "utf-8"   # keep UTF-8 for accents

# --- Politeness / performance ---
SLEEP_BASE       = 3.5    # base delay between requests
MAX_CANDIDATES   = 25     # max candidate profiles to evaluate per player
CHECKPOINT_EVERY = 20     # write checkpoint every N rows (0 to disable)
START_INDEX      = 0      # resume from this row index
DRY_RUN_LIMIT    = 0      # >0 to process only first N rows for a quick test

# --- Matching requirements ---
REQUIRE_BOTH     = False  # if True, require both team IDs; if False, accept >=1 overlap (preferred: both)
PREFER_BOTH      = True   # still prefer both-team matches when multiple candidates

# --- Discovery strategy ---
# If you set env var BING_API_KEY (or put it here), we'll use Bing first.
BING_API_KEY     = os.getenv("BING_API_KEY", "").strip()

print("Config loaded.")

Config loaded.


In [11]:
# %%
TM_SEARCH_URL = "https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche?query={query}"
BING_ENDPOINT = "https://api.bing.microsoft.com/v7.0/search"

PLAYER_ID_RE = re.compile(r"/spieler/(\d+)(?:/|$)", re.IGNORECASE)
TEAM_ID_RE   = re.compile(r"/verein/(\d+)(?:/|$)",  re.IGNORECASE)

def polite_sleep(base=SLEEP_BASE, jitter=1.2):
    time.sleep(base + random.random() * jitter)

def backoff_wait(attempt, base=1.8, cap=35.0):
    time.sleep(min(cap, (base ** attempt) + random.random()))

def make_session():
    s = requests.Session()
    s.headers.update({
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/124.0.0.0 Safari/537.36"
        ),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    })
    return s

def fetch_html(session: requests.Session, url: str, attempts: int = 4) -> Optional[str]:
    for attempt in range(attempts):
        try:
            r = session.get(url, allow_redirects=True, timeout=25)
            if r.status_code in (429, 500, 502, 503, 520):
                backoff_wait(attempt)
                continue
            r.raise_for_status()
            return r.text
        except requests.RequestException:
            backoff_wait(attempt)
    return None

def extract_player_id_from_url(url: str) -> Optional[int]:
    m = PLAYER_ID_RE.search(url or "")
    if not m:
        return None
    try:
        return int(m.group(1))
    except Exception:
        return None

def extract_team_ids_from_html(html: str) -> Set[int]:
    if not html:
        return set()
    ids = set()
    for m in TEAM_ID_RE.finditer(html):
        try:
            ids.add(int(m.group(1)))
        except Exception:
            pass
    return ids

def as_clean_str(val: object) -> str:
    """Ensure a real Python str ('' if nullish)."""
    if val is None:
        return ""
    try:
        if isinstance(val, float) and pd.isna(val):
            return ""
    except Exception:
        pass
    return str(val).strip()

In [12]:
# %%
def bing_candidate_urls(api_key: str, player_name: str, count: int = 12) -> List[str]:
    """Return candidate Transfermarkt player profile URLs via Bing Web Search."""
    headers = {"Ocp-Apim-Subscription-Key": api_key}
    query = f'site:transfermarkt. "profil/spieler" "{player_name}"'
    params = {"q": query, "count": count, "responseFilter": "Webpages", "safeSearch": "Off"}
    urls = []
    for attempt in range(4):
        try:
            resp = requests.get(BING_ENDPOINT, headers=headers, params=params, timeout=25)
            if resp.status_code == 429:
                backoff_wait(attempt)
                continue
            resp.raise_for_status()
            data = resp.json()
            for item in data.get("webPages", {}).get("value", []):
                url = item.get("url", "")
                # Accept any transfermarkt.* TLD and require /spieler/<id>
                if "transfermarkt." in url and "/spieler/" in url and "/profil/spieler/" in url:
                    urls.append(url)
            break
        except requests.RequestException:
            backoff_wait(attempt)
    return urls

def tm_quick_search_candidate_urls(session: requests.Session, player_name: str, max_candidates: int = 25) -> List[str]:
    """Return candidate profile URLs from Transfermarkt quick search results."""
    url = TM_SEARCH_URL.format(query=quote_plus(player_name))
    html = fetch_html(session, url)
    polite_sleep()
    if not html:
        return []
    soup = BeautifulSoup(html, "html.parser")
    out, seen = [], set()
    # robust selector: anchors to profile pages
    for a in soup.select('a[href*="/profil/spieler/"]'):
        href = a.get("href", "")
        if "/spieler/" not in href:
            continue
        pid = extract_player_id_from_url(href)
        if not pid or pid in seen:
            continue
        seen.add(pid)
        out.append(href if href.startswith("http") else f"https://www.transfermarkt.com{href}")
        if len(out) >= max_candidates:
            break
    return out

def discover_candidates(session: requests.Session, player_name: str) -> List[Tuple[int, str]]:
    """Hybrid discovery: Bing first (if key provided), then TM quick search. Return list of (pid, url)."""
    candidates = []

    # 1) Bing
    if BING_API_KEY:
        for url in bing_candidate_urls(BING_API_KEY, player_name):
            pid = extract_player_id_from_url(url)
            if pid:
                candidates.append((pid, url))

    # 2) TM quick search fallback
    if not candidates:
        for url in tm_quick_search_candidate_urls(session, player_name, max_candidates=MAX_CANDIDATES):
            pid = extract_player_id_from_url(url)
            if pid:
                candidates.append((pid, url))

    # Deduplicate by pid, preserve order
    seen = set()
    unique = []
    for pid, url in candidates:
        if pid in seen:
            continue
        seen.add(pid)
        unique.append((pid, url))
    return unique

In [13]:
# %%
def score_candidate(session: requests.Session, url: str, team_from: int, team_to: int) -> Tuple[int, Set[int]]:
    """
    Fetch player page and return (score, ids_on_page).
    Score is number of overlapping team IDs (0, 1, or 2).
    """
    html = fetch_html(session, url)
    polite_sleep()
    if not html:
        return (0, set())
    ids = extract_team_ids_from_html(html)
    overlap = 0
    if team_from in ids:
        overlap += 1
    if team_to   in ids:
        overlap += 1
    return (overlap, ids)

def resolve_player_id(
    session: requests.Session,
    player_name: str,
    team_from: int,
    team_to: int,
    require_both: bool = REQUIRE_BOTH,
    prefer_both: bool = PREFER_BOTH
) -> Optional[int]:
    """
    - Discover candidates
    - Score each by overlap with team IDs
    - Prefer both-team overlap; accept >=1 if require_both=False
    - If nothing qualifies, fall back to the *first discovered* candidate (very last resort)
    """
    cands = discover_candidates(session, player_name)
    if not cands:
        return None

    best_pid = None
    best_score = -1

    # First pass: evaluate all candidates
    for pid, url in cands:
        score, _ = score_candidate(session, url, team_from, team_to)
        # If require_both, accept immediately only when score == 2
        if require_both:
            if score == 2:
                return pid
            else:
                continue
        # If not require_both: track best
        if score > best_score:
            best_score = score
            best_pid = pid
            # Early exit if we got both teams and prefer_both
            if prefer_both and score == 2:
                break

    # If we had any overlap (>=1), accept best_pid
    if best_score >= 1:
        return best_pid

    # As a last resort (loosen further): accept the first candidate ID, even if 0 overlap.
    # Comment the next line if you don't want this fallback.
    return cands[0][0]

In [14]:
# %%
# Load and validate
df = pd.read_csv(INPUT_PATH, dtype=str, encoding=ENCODING)
required = {"player_name", "transfer_from_id", "transfer_to_id", "tm_player_id"}
missing = required - set(df.columns.astype(str))
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Normalize target column
if "tm_player_id" not in df.columns:
    df["tm_player_id"] = ""

session = make_session()

filled = 0
attempted = 0
unresolved_rows = []  # collect (index, player_name, tf, tt, reason)

pid_col_idx = df.columns.get_loc("tm_player_id")

total_rows = len(df) if DRY_RUN_LIMIT <= 0 else min(DRY_RUN_LIMIT, len(df))
print(f"Processing {total_rows} rows (from index {START_INDEX}). Strategy: "
      f"{'Bing+TM' if BING_API_KEY else 'TM quick search only'}, "
      f"require_both={REQUIRE_BOTH}, prefer_both={PREFER_BOTH}")

for pos, row in enumerate(df.itertuples(index=True), start=0):
    if pos < START_INDEX:
        continue
    if pos >= total_rows:
        break

    idx = row.Index
    current = as_clean_str(getattr(row, "tm_player_id"))
    if current:
        continue

    name = as_clean_str(getattr(row, "player_name"))
    tf_s = as_clean_str(getattr(row, "transfer_from_id"))
    tt_s = as_clean_str(getattr(row, "transfer_to_id"))

    # parse team IDs
    try:
        tf = int(tf_s)
        tt = int(tt_s)
    except ValueError:
        unresolved_rows.append((idx, name, tf_s, tt_s, "non-integer team id(s)"))
        continue

    attempted += 1

    try:
        pid = resolve_player_id(
            session=session,
            player_name=name,
            team_from=tf,
            team_to=tt,
            require_both=REQUIRE_BOTH,
            prefer_both=PREFER_BOTH
        )
    except Exception as e:
        pid = None

    if pid is not None:
        df.iat[idx, pid_col_idx] = str(pid)
        filled += 1
        if filled % 10 == 0:
            print(f"Filled {filled} so far (last: {name} -> {pid})")
    else:
        unresolved_rows.append((idx, name, tf, tt, "no candidate / score<min"))

    if CHECKPOINT_EVERY and ((pos + 1) % CHECKPOINT_EVERY == 0):
        ckpt = f"{os.path.splitext(OUTPUT_PATH)[0]}__checkpoint.csv"
        df.to_csv(ckpt, index=False, encoding=ENCODING)
        print(f"[Checkpoint] wrote {ckpt} at row {pos+1}")

print(f"Done. Attempted {attempted}, filled {filled}.")

Processing 89 rows (from index 0). Strategy: TM quick search only, require_both=False, prefer_both=True
Filled 10 so far (last: Nery Castillo -> 9711)
[Checkpoint] wrote mls_dps_with_tm__checkpoint.csv at row 20
Filled 20 so far (last: Oswaldo Minda -> 82718)
Filled 30 so far (last: Branko Bošković -> 9090)
[Checkpoint] wrote mls_dps_with_tm__checkpoint.csv at row 40
Filled 40 so far (last: Ondřej Lingr -> 321195)
Filled 50 so far (last: Ménder García -> 585467)
[Checkpoint] wrote mls_dps_with_tm__checkpoint.csv at row 60
Filled 60 so far (last: Alexandru Mitriță -> 209649)
Filled 70 so far (last: Fanendo Adi -> 79654)
[Checkpoint] wrote mls_dps_with_tm__checkpoint.csv at row 80
Filled 80 so far (last: Hernán López -> 665309)
Done. Attempted 89, filled 84.


In [15]:
# %%
df.to_csv(OUTPUT_PATH, index=False, encoding=ENCODING)
print(f"Saved updated file to: {OUTPUT_PATH}")

unresolved_df = pd.DataFrame(unresolved_rows, columns=["row_index", "player_name", "transfer_from_id", "transfer_to_id", "reason"])
print(f"Unresolved rows: {len(unresolved_df)}")
display(unresolved_df.head(20))


Saved updated file to: mls_dps_with_tm.csv
Unresolved rows: 5


,row_index,player_name,transfer_from_id,transfer_to_id,reason
0,3,Saba Lobzhanidze,7775,51663,no candidate / score<min
1,48,Son Heung-min,148,51828,no candidate / score<min
2,61,Maximiliano Moralez,4941,40058,no candidate / score<min
3,85,Mustapha Jarju,1411,6321,no candidate / score<min
4,87,Hwang In-beom,6499,6321,no candidate / score<min
